In [1]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

"""
This notebook just does everything in ./sh/preprocess.sh script
in an interactive manner.
"""

import sys
import os

sys.path.append('../')
sys.path.append('../paretoviz')
from paretoviz.utils import fmt
from paretoviz.utils import vectorops as vops

from paretoviz import normalizer as nrm
from paretoviz import tradeoff_calculator as tro
from paretoviz import peeler as plr
from paretoviz import palettizer as pltz

# Do all these plots with centroid based coloring

# rawfpath = "../data/spherical/spherical-3d.out"
# rawfpath = "../data/spherical/spherical-4d.out"
# rawfpath = "../data/spherical/spherical-8d.out"

# rawfpath = "../data/knee/knee-3d.out"
# rawfpath = "../data/knee/knee-4d.out"
# rawfpath = "../data/knee/knee-8d.out"

# rawfpath = "../data/line-surface/line-3d.out"
# rawfpath = "../data/line-surface/line-4d.out"
# rawfpath = "../data/line-surface/line-6d.out" # fails
# rawfpath = "../data/line-surface/line-8d.out"

# Do all these plots with constraint based coloring

# rawfpath = "../data/knee-const/knee-const-3d.out"
# rawfpath = "../data/knee-const/knee-const-4d.out"
# rawfpath = "../data/knee-const/knee-const-8d.out"

# rawfpath = "../data/knee-const-mod/knee-const-mod-3d.out"
# rawfpath = "../data/knee-const-mod/knee-const-mod-4d.out"

# rawfpath = "../data/isolated/isolated-3d.out"
# rawfpath = "../data/isolated/isolated-4d.out"
# rawfpath = "../data/isolated/isolated-8d.out"

# rawfpath = "../data/c2dtlz2/c2dtlz2-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-4d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-5d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-8d.out"

# rawfpath = "../data/c2dtlz2/c2dtlz2-c1/c2dtlz2-c1-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c2/c2dtlz2-c2-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c3/c2dtlz2-c3-3d.out"
# rawfpath = "../data/c2dtlz2/c2dtlz2-c4/c2dtlz2-c4-3d.out"

rawfpath = "../data/osy/osy-3d.out"
# rawfpath = "../data/osy/osy-4d.out"

# some engineering design problems

# rawfpath = "../data/carcrash/carcrash-3d.out"
# rawfpath = "../data/carcrash/carcrash-c1/carcrash-c1-3d.out"
# rawfpath = "../data/carcrash/carcrash-c2/carcrash-c2-3d.out"

# rawfpath = "../data/gaa-das/gaa-das-10d.out"
# rawfpath = "../data/gaa-lhs/gaa-lhs-10d.out"

# some ML data sets

# rawfpath = "../data/airofoil/airofoil-5d.out"
# rawfpath = "../data/banknote/banknote-4d.out"
# rawfpath = "../data/blood/blood-4d.out" # fails
# rawfpath = "../data/cccp/cccp-4d.out"
# rawfpath = "../data/concrete/concrete-8d.out" # fails
# rawfpath = "../data/iris/iris-4d.out" 
# rawfpath = "../data/mammogram/mammogram-5d.out" # fails
# rawfpath = "../data/wil/wil-7d.out" # fails
# rawfpath = "../data/yeast/yeast-8d.out" #  fails 

path, rawfname = os.path.split(rawfpath)
normfname = rawfname.split('.')[0] + "-norm.out"
mufname = normfname.split('.')[0] + "-mu.out"

In [2]:
# This part does the normalizer.py operation

vals = fmt.load(rawfpath)

print("Normalizing {0:d} data points.".format(len(vals)))
vals_ = nrm.normalize(vals)
fmt.cat(vals_)

path, rawfile = os.path.split(rawfpath)
normfile = os.path.join(path, rawfile.split('.')[0] + "-norm.out")
print("Saving normalized data into {0:s} ...".format(normfile))
fmt.save(vals_, normfile)

Normalizing 2000 data points.
0.0000e+00	4.2219e-01	9.9995e-01
3.1243e-03	4.1682e-01	1.0000e+00
1.0000e+00	0.0000e+00	6.6000e-03
6.5006e-01	7.7255e-02	0.0000e+00
7.2055e-02	1.0000e+00	9.6165e-01
5.3988e-01	4.7812e-01	6.6080e-01
5.8872e-01	2.3274e-01	8.8000e-03
9.9930e-01	1.1738e-06	6.6500e-03
9.5063e-01	6.9843e-05	8.0000e-04
8.9532e-01	2.9627e-03	9.0000e-04
8.5031e-01	3.0285e-03	1.0000e-04
8.1756e-01	5.6109e-03	4.3500e-03
7.8716e-01	5.7582e-03	4.5000e-04
7.2505e-02	9.9923e-01	9.6165e-01
6.8152e-02	9.8255e-01	9.6390e-01
7.0000e-01	1.0898e-02	1.4500e-03
8.4681e-01	3.5268e-03	3.0000e-04
7.5557e-01	7.2320e-03	5.0000e-05
7.7085e-01	6.8874e-03	1.5000e-04
6.8484e-01	1.1047e-02	1.4000e-03
8.3233e-01	3.7897e-03	1.5000e-04
8.1903e-01	4.3238e-03	2.0000e-04
8.3227e-01	5.3298e-03	1.5000e-04
7.7085e-01	6.4995e-03	1.5000e-04
7.8355e-01	6.9890e-03	5.0000e-05
9.3408e-01	3.7680e-04	5.0000e-05
9.0746e-01	2.6728e-03	7.5000e-04
8.9538e-01	1.4937e-03	9.0000e-04
9.1670e-01	9.2322e-04	1.9500e-03
9.2395e-01	6.

In [3]:
# This part does the tradeoff_calculator.py operation

# epsilon = 0.33 # spherical
# epsilon = 0.25 # knee
# epsilon = 0.025 # line
# epsilon = 0.125 # knee-const, mod
# epsilon = 0.33 # isolated-3d, 4d
# epsilon = 0.025 # isolated-8d
# epsilon = 0.33 # c2dtlz2-3d, 4d
# epsilon = 0.15 # c2dtlz2-5d
# epsilon = 0.25 # carcrash
epsilon = 0.33 # osy

normfpath = os.path.join(path, normfname)
if os.path.exists(normfpath):
    points = fmt.load(normfpath)

    mu = tro.compute_tradeoff(points, epsilon, normalize = False)
    fmt.cat(mu)

    path, normfname = os.path.split(normfpath)
    mufpath = os.path.join(path, normfname.split('.')[0] + "-mu.out")
    print("Saving tradeoff values to {0:s} ...".format(mufpath))
    fmt.save(mu, mufpath)
else:
    print("Error: " + normfpath + " does not exist.")

Computing neighborhood ...
Computing tradeoff values with epsilon = 0.3300...
5.5175e-01
5.5041e-01
1.2660e-03
4.0493e-01
4.7023e-03
3.3591e-02
1.4061e-01
1.2595e-03
1.2516e-02
1.4364e-03
4.2882e-02
4.2945e-03
3.3684e-02
2.5703e-03
1.4306e-03
9.7962e-03
1.7970e-02
5.0944e-02
2.4155e-05
3.8866e-02
4.3039e-02
4.4055e-02
3.5098e-03
4.7627e-02
7.5818e-03
2.9269e-02
6.1540e-03
3.3460e-02
2.5502e-02
3.2662e-02
1.2311e-02
2.1445e-02
1.2227e-02
3.9704e-02
4.6647e-03
8.4269e-03
3.9324e-02
4.7944e-02
3.3552e-03
3.3178e-02
2.4155e-05
2.4155e-05
3.5177e-03
4.1208e-03
8.7517e-03
4.2918e-03
4.5413e-04
4.4792e-03
6.9279e-03
1.8470e-02
1.7004e-02
1.1811e-03
5.6844e-04
4.8152e-03
1.8653e-02
8.6467e-03
1.6696e-03
5.1356e-03
4.5290e-03
3.6948e-03
2.4155e-05
3.1317e-04
2.4155e-05
3.2545e-04
2.4155e-05
2.4155e-05
1.1953e-03
5.0055e-03
5.3227e-02
2.4012e-03
4.9381e-02
1.8633e-03
3.6755e-03
6.1577e-03
1.4523e-03
1.0415e-03
1.3412e-03
1.1978e-03
2.7419e-03
2.1501e-02
8.9812e-02
5.6471e-02
3.2928e-04
2.5132e-0

In [4]:
# This part does the peeler.py operation

mode = "default"
# mode = "no-project" # use it for line-surface

normfpath = os.path.join(path, normfname)
points = fmt.load(normfpath)

m = len(points[0])
print("Peeling data point cloud in {0:s} mode ...".format(mode))
if mode == "default":
    # the defualt mode will project the points 
    # on a simplex on the first quadrant
    ppoints = plr.project(points)
    cpoints = plr.collapse(ppoints, dim = m - 1)
    boundaries = plr.peel(cpoints)
elif mode == "no-project":
    boundaries = plr.peel(points)
fmt.cat(boundaries, dtype = 'int')

layerfpath = os.path.join(path, normfname.split('.')[0] + "-layers.out")
print("Saving {0:d} layers into {1:s} ...".format(len(boundaries), layerfpath))
fmt.save(boundaries, layerfpath, dtype = 'int')

Peeling data point cloud in default mode ...
Total points: 2000
Computing alpha-shape ...
alpha-shape done.
Layer 0: points added 17, points left 1983.
Computing alpha-shape ...
alpha-shape done.
Layer 1: points added 17, points left 1966.
Computing alpha-shape ...
alpha-shape done.
Layer 2: points added 26, points left 1940.
Computing alpha-shape ...
alpha-shape done.
Layer 3: points added 25, points left 1915.
Computing alpha-shape ...
alpha-shape done.
Layer 4: points added 30, points left 1885.
Computing alpha-shape ...
alpha-shape done.
Layer 5: points added 27, points left 1858.
Computing alpha-shape ...
alpha-shape done.
Layer 6: points added 26, points left 1832.
Computing alpha-shape ...
alpha-shape done.
Layer 7: points added 24, points left 1808.
Computing alpha-shape ...
alpha-shape done.
Layer 8: points added 30, points left 1778.
Computing alpha-shape ...
alpha-shape done.
Layer 9: points added 24, points left 1754.
Computing alpha-shape ...
alpha-shape done.
Layer 10: po

In [6]:
# This part does the palettizer.py operation

n_layers = 4
mode = "invsc"
# mode = "invrv"
# mode = "rv"
# mode = "sc"

normfpath = os.path.join(path, normfname)
points = fmt.load(normfpath)

layerfile = normfname.split('.')[0] + "-layers.out"
layers = fmt.load(os.path.join(path, layerfile), dtype = 'int')

print("Computing palette coordinates with {0:d} layers in {1:s} mode ...".format(n_layers, mode))
if mode == "invsc":
    palette_coords = pltz.palettize_sc(points, layers, n_layers = n_layers, inverted = True)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-invsc.out")
elif mode == "invrv":
    palette_coords = pltz.palettize_rv(points, layers, n_layers = n_layers, inverted = True)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-invrv.out")
elif mode == "rv":
    palette_coords = pltz.palettize_rv(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-rv.out")
elif mode == "sc":
    palette_coords = pltz.palettize_sc(points, layers, n_layers = n_layers)
    palettefpath = os.path.join(path, normfname.split('.')[0] + "-palette-sc.out")
else:
    print("Error: unknown mode \'{0:s}\'\n".format(mode))
    sys.exit(1)

print("Saving palette coordinates into {0:s} ...".format(palettefpath))
pltz.save_palette(palette_coords, palettefpath)

Computing palette coordinates with 4 layers in invsc mode ...
Saving palette coordinates into ../data/osy/osy-3d-norm-palette-invsc.out ...
